In [ ]:
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

def process_cell_counts(i, cell_positions, cell_labels, dist_bin_px):
    # squared distance
    counts = np.sum(np.square(cell_positions[i][np.newaxis, :] - cell_positions), axis=1)
    # inequalities around arcs
    counts = counts[np.newaxis, :] <= np.square(np.concatenate([[0], dist_bin_px]))[:, np.newaxis]
    # matmul to counts
    counts = np.diff(np.matmul(counts.astype(int), cell_labels.astype(int)), axis=0)
    # return index and counts
    return i, counts

def euclidian_counts(idx, distances, cell_labels, dist_bin_px):

    idx_counts = None
    cell_labels_uni = sorted(cell_labels.unique())
    dist_bin_px = np.concatenate([[0], dist_bin_px])
    for i in range(len(dist_bin_px)-1):
        present_cells = cell_labels[(distances[idx] > dist_bin_px[i]) & (distances[idx] <= dist_bin_px[i+1])]
        these_counts = [sum(present_cells == label) for label in cell_labels_uni]

        if idx_counts is not None:
            idx_counts = np.vstack((idx_counts, these_counts))
        else:
            idx_counts = np.array(these_counts)

    return idx_counts

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def draw_cell_scatter(df, ind = None):
    sns.scatterplot(data = df,
                    x = 'Cell X Position',
                    y = 'Cell Y Position',
                    hue = 'Lineage')
    
    if ind is not None:
        plt.Circle(df.loc[ind, ['Cell X Position', 'Cell Y Position']])

In [ ]:
import pandas as pd
from functools import partial

dist_bin_um=np.array([25, 50, 100, 150, 200])
dist_bin_px = dist_bin_um/ 0.5

filename = './sample.csv'
df = pd.read_csv(filename)

In [ ]:
distances = euclidean_distances(df[['Cell X Position', 'Cell Y Position']])
cell_labels = df['Lineage']

args = dict(distances=distances,
            cell_labels=cell_labels,
            dist_bin_px=dist_bin_px)
pool_map_fn = partial(euclidian_counts, **args)

idxSet = range(10)
result = list(map(lambda x: np.stack(x, axis=0), list(map(pool_map_fn, idxSet))))


In [ ]:
cell_positions = df[['Cell X Position', 'Cell Y Position']].values
cell_labels = pd.get_dummies(df['Lineage'])

args = dict(cell_positions=cell_positions,
            cell_labels=cell_labels.values,
            dist_bin_px=dist_bin_px)
pool_map_fn = partial(process_cell_counts, **args)

idxSet = range(100)
i, counts = list(map(lambda x: np.stack(x, axis=0), list(zip(*map(pool_map_fn, idxSet)))))